In [1]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import asari
import warnings
from asari.api import Sonar


In [2]:
df = pd.read_csv("myboysuga.csv")
df.columns
df.head()

,date,time,username,name,tweet,replies_count,retweets_count,likes_count
0,2021-02-02,05:19:01,sugawitter,菅 義偉,引き続き、制約の多い生活でご苦労をおかけしますが、お一人お一人が行動を変えていただくことで、...,433,385,3276
1,2021-02-02,05:18:21,sugawitter,菅 義偉,新型コロナウイルスの発生から1年が経ちました。日本でも世界でも、ウイルスとの闘いは続いていま...,99,329,2933
2,2021-02-02,05:17:03,sugawitter,菅 義偉,重層的なセーフティーネットにより事業を守り、雇用と暮らしを守り、困難を抱えた方々を支えていき...,62,225,1192
3,2021-02-02,05:16:14,sugawitter,菅 義偉,ここで、感染の減少傾向を確かなものにしなければならない、そう判断いたしました。 飲食店の時...,56,242,1309
4,2021-02-02,05:15:17,sugawitter,菅 義偉,また先般、内閣の一員である副大臣やわが党に所属した議員が深夜まで会食した問題はあってはならな...,93,305,1644


In [3]:
#One of the more interesting tweets that I've read recently. 
df["tweet"][4]

'また先般、内閣の一員である副大臣やわが党に所属した議員が深夜まで会食した問題はあってはならないことであります。素直にお詫びを申し上げます。  皆様のご協力により、全国の感染者数は大幅に減少してきており、対策の効果がはっきりと出始めています。'

In [4]:
#Data frame with translated tweets. 
df2 = pd.read_csv("myboysuga2.csv")
df2.columns
df2.head()

,date,time,username,name,tweet,tweet_en,replies_count,retweets_count,likes_count
0,2021-02-02,5:19:01,sugawitter,菅 義偉,引き続き、制約の多い生活でご苦労をおかけしますが、お一人お一人が行動を変えていただくことで、...,"Continue, We apologize for the hardships in th...",433.0,385.0,3276.0
1,2021-02-02,5:18:21,sugawitter,菅 義偉,新型コロナウイルスの発生から1年が経ちました。日本でも世界でも、ウイルスとの闘いは続いていま...,1 year from the occurrence of the new coronavi...,99.0,329.0,2933.0
2,2021-02-02,5:17:03,sugawitter,菅 義偉,重層的なセーフティーネットにより事業を守り、雇用と暮らしを守り、困難を抱えた方々を支えていき...,Protect the business through multi-level safet...,62.0,225.0,1192.0
3,2021-02-02,5:16:14,sugawitter,菅 義偉,ここで、感染の減少傾向を確かなものにしなければならない、そう判断いたしました。 飲食店の時...,"Here, must be to ensure its downward trend of ...",56.0,242.0,1309.0
4,2021-02-02,5:15:17,sugawitter,菅 義偉,また先般、内閣の一員である副大臣やわが党に所属した議員が深夜まで会食した問題はあってはならな...,"Also recently, a problem that lawmakers who be...",93.0,305.0,1644.0


In [5]:
date_regex = df.date.str.split("-", expand= True)
date_regex.columns = ["year", "month", "day"]
date_regex.head()

,year,month,day
0,2021,02,02
1,2021,02,02
2,2021,02,02
3,2021,02,02
4,2021,02,02


In [6]:
df = pd.concat([df, date_regex], axis = 1)
df.head()

,date,time,username,name,tweet,replies_count,retweets_count,likes_count,year,month,day
0,2021-02-02,05:19:01,sugawitter,菅 義偉,引き続き、制約の多い生活でご苦労をおかけしますが、お一人お一人が行動を変えていただくことで、...,433,385,3276,2021,02,02
1,2021-02-02,05:18:21,sugawitter,菅 義偉,新型コロナウイルスの発生から1年が経ちました。日本でも世界でも、ウイルスとの闘いは続いていま...,99,329,2933,2021,02,02
2,2021-02-02,05:17:03,sugawitter,菅 義偉,重層的なセーフティーネットにより事業を守り、雇用と暮らしを守り、困難を抱えた方々を支えていき...,62,225,1192,2021,02,02
3,2021-02-02,05:16:14,sugawitter,菅 義偉,ここで、感染の減少傾向を確かなものにしなければならない、そう判断いたしました。 飲食店の時...,56,242,1309,2021,02,02
4,2021-02-02,05:15:17,sugawitter,菅 義偉,また先般、内閣の一員である副大臣やわが党に所属した議員が深夜まで会食した問題はあってはならな...,93,305,1644,2021,02,02


In [7]:
warnings.filterwarnings('ignore')

sonar = Sonar()

def positive_sentiment(text):
    info = sonar.ping(text= text)
    positive_vector = info["classes"][1]["confidence"]
    return positive_vector


def negative_sentiment(text):
    info = sonar.ping(text= text)
    negative_vector = info["classes"][0]["confidence"]
    return negative_vector


def which_sentiment(text):
    info = sonar.ping(text= text)
    sentiment = info["top_class"]
    return sentiment


In [15]:
#Testing if sonar function works 
sonar = Sonar()
sonar.ping(text="広告多すぎる♡")


{'text': 'I hate that I love you',
 'top_class': 'positive',
 'classes': [{'class_name': 'negative', 'confidence': 0.01851598048197674},
  {'class_name': 'positive', 'confidence': 0.9814840195180232}]}

In [10]:
df["positive_sentiment"] = df.tweet.map(lambda x : positive_sentiment(x))


df["negative_sentiment"] = df.tweet.map(lambda x : negative_sentiment(x))


df["sentiment"] = df.tweet.map(lambda x : which_sentiment(x))


df["net_sentiment"] = df.positive_sentiment - df.negative_sentiment


In [11]:
df.head()

,date,time,username,name,tweet,replies_count,retweets_count,likes_count,year,month,day,positive_sentiment,negative_sentiment,sentiment,net_sentiment
0,2021-02-02,05:19:01,sugawitter,菅 義偉,引き続き、制約の多い生活でご苦労をおかけしますが、お一人お一人が行動を変えていただくことで、...,433,385,3276,2021,02,02,0.927572,0.072428,positive,0.855145
1,2021-02-02,05:18:21,sugawitter,菅 義偉,新型コロナウイルスの発生から1年が経ちました。日本でも世界でも、ウイルスとの闘いは続いていま...,99,329,2933,2021,02,02,0.994129,0.005871,positive,0.988258
2,2021-02-02,05:17:03,sugawitter,菅 義偉,重層的なセーフティーネットにより事業を守り、雇用と暮らしを守り、困難を抱えた方々を支えていき...,62,225,1192,2021,02,02,0.902652,0.097348,positive,0.805305
3,2021-02-02,05:16:14,sugawitter,菅 義偉,ここで、感染の減少傾向を確かなものにしなければならない、そう判断いたしました。 飲食店の時...,56,242,1309,2021,02,02,0.562818,0.437182,positive,0.125635
4,2021-02-02,05:15:17,sugawitter,菅 義偉,また先般、内閣の一員である副大臣やわが党に所属した議員が深夜まで会食した問題はあってはならな...,93,305,1644,2021,02,02,0.898700,0.101300,positive,0.797400


In [18]:
df2["positive_sentiment"] = df2.tweet_en.map(lambda x : positive_sentiment(x))


df2["negative_sentiment"] = df2.tweet_en.map(lambda x : negative_sentiment(x))


df2["sentiment"] = df2.tweet_en.map(lambda x : which_sentiment(x))


df2["net_sentiment"] = df2.positive_sentiment - df2.negative_sentiment

In [19]:
df2.head()

,date,time,username,name,tweet,tweet_en,replies_count,retweets_count,likes_count,positive_sentiment,negative_sentiment,sentiment,net_sentiment
0,2021-02-02,5:19:01,sugawitter,菅 義偉,引き続き、制約の多い生活でご苦労をおかけしますが、お一人お一人が行動を変えていただくことで、...,"Continue, We apologize for the hardships in th...",433.0,385.0,3276.0,0.900866,0.099134,positive,0.801733
1,2021-02-02,5:18:21,sugawitter,菅 義偉,新型コロナウイルスの発生から1年が経ちました。日本でも世界でも、ウイルスとの闘いは続いていま...,1 year from the occurrence of the new coronavi...,99.0,329.0,2933.0,0.806520,0.193480,positive,0.613040
2,2021-02-02,5:17:03,sugawitter,菅 義偉,重層的なセーフティーネットにより事業を守り、雇用と暮らしを守り、困難を抱えた方々を支えていき...,Protect the business through multi-level safet...,62.0,225.0,1192.0,0.909216,0.090784,positive,0.818431
3,2021-02-02,5:16:14,sugawitter,菅 義偉,ここで、感染の減少傾向を確かなものにしなければならない、そう判断いたしました。 飲食店の時...,"Here, must be to ensure its downward trend of ...",56.0,242.0,1309.0,0.835951,0.164049,positive,0.671901
4,2021-02-02,5:15:17,sugawitter,菅 義偉,また先般、内閣の一員である副大臣やわが党に所属した議員が深夜まで会食した問題はあってはならな...,"Also recently, a problem that lawmakers who be...",93.0,305.0,1644.0,0.801945,0.198055,positive,0.603891
